In [33]:
import pandas as pd
import numpy as np
import math 
import matplotlib.pyplot as plt 
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import  make_scorer
from sklearn.externals import joblib

In [34]:
#cargo los datos en el dataframe
dataframe=pd.read_csv('/content/100-amk-343.csv') # cargar archivo generado por tablaAlelosAB.py 
dataframe = dataframe.drop(['id', 'aislamiento', 'AMKI'], axis='columns') #AMKI hace referencia a la columna en donde esta clasificado el aislamiento como sensible, intermedio, resistente. En caso que contengan esa columna eliminar AMKI
print(dataframe)

     OXA-64  rosA  rosA_cp1  macB  ...  OXA-104  OXA-132  OXA-180  AMIKACINA
0       1.0   8.0      -1.0   1.0  ...     -1.0     -1.0     -1.0        128
1       1.0   8.0      -1.0   1.0  ...     -1.0     -1.0     -1.0         64
2      -1.0   5.0      -1.0   4.0  ...     -1.0     -1.0     -1.0         32
3      -1.0   5.0      -1.0   4.0  ...     -1.0     -1.0     -1.0         32
4      -1.0   5.0      -1.0   4.0  ...     -1.0     -1.0     -1.0         32
..      ...   ...       ...   ...  ...      ...      ...      ...        ...
337    -1.0  29.0      -1.0   1.0  ...     -1.0      1.0     -1.0         32
338    -1.0   5.0      -1.0   4.0  ...     -1.0     -1.0     -1.0         64
339    -1.0  28.0      -1.0  11.0  ...     -1.0     -1.0      1.0         64
340    -1.0  27.0      -1.0   1.0  ...     -1.0     -1.0     -1.0         16
341    -1.0   2.0      -1.0   2.0  ...     -1.0     -1.0     -1.0         64

[342 rows x 212 columns]


In [35]:
#defino las variables "X" y "y"
X = dataframe.drop(['AMIKACINA'], axis='columns')
y = dataframe.iloc[:, -1] 
print(X)
print(y)

     OXA-64  rosA  rosA_cp1  macB  ...  OXA-431  OXA-104  OXA-132  OXA-180
0       1.0   8.0      -1.0   1.0  ...     -1.0     -1.0     -1.0     -1.0
1       1.0   8.0      -1.0   1.0  ...     -1.0     -1.0     -1.0     -1.0
2      -1.0   5.0      -1.0   4.0  ...     -1.0     -1.0     -1.0     -1.0
3      -1.0   5.0      -1.0   4.0  ...     -1.0     -1.0     -1.0     -1.0
4      -1.0   5.0      -1.0   4.0  ...     -1.0     -1.0     -1.0     -1.0
..      ...   ...       ...   ...  ...      ...      ...      ...      ...
337    -1.0  29.0      -1.0   1.0  ...     -1.0     -1.0      1.0     -1.0
338    -1.0   5.0      -1.0   4.0  ...     -1.0     -1.0     -1.0     -1.0
339    -1.0  28.0      -1.0  11.0  ...     -1.0     -1.0     -1.0      1.0
340    -1.0  27.0      -1.0   1.0  ...     -1.0     -1.0     -1.0     -1.0
341    -1.0   2.0      -1.0   2.0  ...     -1.0     -1.0     -1.0     -1.0

[342 rows x 211 columns]
0      128
1       64
2       32
3       32
4       32
      ... 
337     

In [36]:
#cambio de variables a categoricas por dummys 
datadummy = pd.get_dummies(X.astype(str))
print(datadummy)

     OXA-64_-1.0  OXA-64_1.0  rosA_-1.0  ...  OXA-132_1.0  OXA-180_-1.0  OXA-180_1.0
0              0           1          0  ...            0             1            0
1              0           1          0  ...            0             1            0
2              1           0          0  ...            0             1            0
3              1           0          0  ...            0             1            0
4              1           0          0  ...            0             1            0
..           ...         ...        ...  ...          ...           ...          ...
337            1           0          0  ...            1             1            0
338            1           0          0  ...            0             1            0
339            1           0          0  ...            0             0            1
340            1           0          0  ...            0             1            0
341            1           0          0  ...            0        

In [37]:
#Eliminar columnas que indican la ausencia del gen 
for col in datadummy.columns: 
  if col[-4:] == '-1.0':
    print(col)
    del(datadummy[col])

print(datadummy.columns)
print(datadummy.shape)

OXA-64_-1.0
rosA_-1.0
rosA_cp1_-1.0
abeM_-1.0
Acinetobacter_baumannii_AmvA_-1.0
Acinetobacter_baumannii_AmvA_cp1_-1.0
adeN_-1.0
adeH_-1.0
adeF_-1.0
adeF_cp1_-1.0
adeL_-1.0
abeS_-1.0
abeS_cp1_-1.0
ANT(3'')-IIc_-1.0
ANT(3'')-IIc_cp1_-1.0
Acinetobacter_baumannii_AbaQ_-1.0
adeB_-1.0
adeB_cp1_-1.0
adeA_-1.0
adeA_cp1_-1.0
adeR_-1.0
adeR_cp1_-1.0
adeS_-1.0
adeS_cp1_-1.0
ADC-5_-1.0
MexK_-1.0
Mycobacterium_tuberculosis_rpoB_mutants_conferring_resistance_to_rifampicin_-1.0
adeI_-1.0
adeJ_-1.0
Clostridioides_difficile_gyrB_conferring_resistance_to_fluoroquinolone_-1.0
Clostridioides_difficile_gyrB_conferring_resistance_to_fluoroquinolone_cp1_-1.0
Escherichia_coli_soxR_with_mutation_conferring_antibiotic_resistance_-1.0
Acinetobacter_baumannii_AbaF_-1.0
sul2_-1.0
sul2_cp1_-1.0
sul2_cp2_-1.0
tet(B)_-1.0
APH(6)-Id_-1.0
APH(6)-Id_cp1_-1.0
APH(3'')-Ib_-1.0
APH(3'')-Ib_cp1_-1.0
OXA-23_-1.0
OXA-23_cp1_-1.0
AAC(3)-IIe_-1.0
AAC(6')-Ian_-1.0
TEM-1_-1.0
Escherichia_coli_gyrA_conferring_resistance_to_fluoroq

In [38]:
#convierto la variable de "y" en log2

y_test1 = np.log2(y)
print(y_test1)


0      7.0
1      6.0
2      5.0
3      5.0
4      5.0
      ... 
337    5.0
338    6.0
339    6.0
340    4.0
341    6.0
Name: AMIKACINA, Length: 342, dtype: float64


In [29]:
#hacer prediccion 
Lasso_from_joblib = joblib.load('/content/amk-100-rl.pkl') #cargar archivo pkl correspondiente al modelo que desea probar
print("Predicciones")
predictions = 2**np.round(Lasso_from_joblib.predict(datadummy))
print(predictions)
verdaderos = y.to_numpy()
print(verdaderos)
# Por último, calculamos el error cuadrático medio y el estadístico R^2
print ('Error cuadrático medio: %.2f' % mean_squared_error(y_test1, np.log2(predictions)))
print ('Estadístico R_2: %.2f' % r2_score(y_test1, np.log2(predictions)))
print(mean_absolute_error(y_test1, np.log2(predictions)))

Predicciones
[ 32.  32.  64.  64.  16.  64.  16.  64.  32.  64.  64. 256.  64.   8.
  64.  64.  16.  16.  32.  64.  64.  64.  32.  32.  16.  16.  64.  32.
  16.  64.  32.  64.  16.   8.  64.  64.  32.  32.  64.  32. 128.  64.
  64.  16.  16.  16.  64.  64.   8.  32.  16.   8.  64.  64. 128.  64.
 128.  64.  64.  16.  16.  32. 128.  32.  64.  16.  32.  32.  16.  16.
  64. 128.  64.  64.  16.  32.  16.  32.  32.  16.  32.  64.   8.  64.
  32.  32.  32.  32.  16.  32.  64.  16.  16.  32.   8.  64.  32.  32.
  16.   8.  32.   8.  64.  16.  32.   8.  32.   8.   8.   8.  32.   8.
   8.  32.  32.   8.   8.  16.  16.   8.  32.  32.  64.   8.   8.  32.
  32.  16.  32.   8.   8.   8.  32.  32.   8.  16.  16.  32.  32.  16.
  16.  16.  32.  32.   8.  16.   8.  32.  32.   8.  16.   8.  32.   8.
  64.  32.  32.   8.   8.  32.  32.  32.   8.  16.  64.   8.   8.  32.
   8.  32.  32.  32.  32.  32.  16.  32.  16.  32.  64.  32.  32.   8.
   8.  32.  32.  32.  32.  16.   8.  32.  16.   8.  32.  16.  32

In [30]:
#funcion para calcular la excatitud con +/- una dilucion 
def within_dillution(y, predictions, dill=2):
    frac = np.array(y) / np.array(predictions)
    return ((frac >= 1/dill) & (frac <= dill)).mean()
   

In [31]:
acc2 = within_dillution(y, predictions, dill=2)
acc4 = within_dillution(y, predictions, dill=4)
print(acc2)
print(acc4)

0.8976608187134503
0.9970760233918129


In [32]:
threshold = 16 # estos valores correspondientes a los puntos de corte del para clasificar como suceptible un genoma (varia segun antibiotico)
threshold1 = 32 # valor correspondiente a los puntos de corte para clasificar como intermedio en el caso que amerite, caso contrario colocamos suceptible
predDF = pd.DataFrame({'predictions':predictions, 'y':y})
vme = len(predDF[(predDF.predictions<=threshold) & (predDF.y>threshold1)])
me = len(predDF[(predDF.predictions>threshold1) & (predDF.y<=threshold)])
print(vme)
print(me)

5
2
